In [1]:
import numpy as np 
import pandas as pd 
import os.path as op  
from datetime import date 
import time 

# The default data directory will be '../../data' 
# This can be redefined by the user of the function. 
data_dir = op.join('data') 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
import massflow_baseline

In [3]:
raw = massflow_baseline.get_data()

In [4]:
question_16 = raw[raw.QuestionNumber == 'Q016']
test = question_16[question_16['ColText'].str.contains("Tonnage")]
test.ColText.unique()

array([u'Tonnage collected for recycling', u'Tonnage collected for reuse',
       u'Tonnage collected for recycling but actually rejected / disposed',
       u'Tonnage collected for reuse but actually rejected / disposed'], dtype=object)

In [6]:
hhkerb_rec_ton_la = massflow_baseline.get_hhkerb_rec_ton_la()
hhkerb_rec_ton_la

RowText,Authority,Aluminium cans,Co mingled materials,Composite food and beverage cartons,HDPE [2],Mixed Plastic Bottles,Mixed cans,Mixed glass,Plastics,Steel cans,sum_dry_rec
0,Blaenau Gwent CBC,NaN,7111.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7112.724
1,Bridgend CBC,172.44,NaN,NaN,NaN,NaN,NaN,3206.880,1498.680,513.14,10763.070
2,Caerphilly CBC,NaN,17828.170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17828.170
3,Cardiff County Council,NaN,32416.590,NaN,8.98,NaN,NaN,NaN,NaN,NaN,32774.610
4,Carmarthenshire County Council,NaN,13012.690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13012.690
5,Ceredigion County Council,NaN,5418.420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5424.040
6,City and County of Swansea,NaN,15855.870,NaN,NaN,NaN,NaN,NaN,3599.840,NaN,19455.710
7,Conwy CBC,NaN,NaN,NaN,NaN,NaN,547.92,3119.250,1066.460,NaN,9171.100
8,Denbighshire County Council,NaN,9720.210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9720.210
9,Flintshire County Council,165.22,12.240,NaN,NaN,1163.00,NaN,3749.760,NaN,413.90,10907.810


#REUSE
All the tonnages recorded for reuse are from non-DRS sources. So they are only added to the "sum_dry_rec" for LAs with comingled materials to get accurate disaggregated tonnages for DRS.

In [7]:
hwrcs_reu_ton = question_16[(question_16.ColText == 'Tonnage collected for reuse') & (question_16.Data > 0)]
hwrcs_reu_ton.RowText.unique()

array([u'Books', u'Textiles & footwear', u'Rubble',
       u'WEEE - Fridges & Freezers', u'WEEE - Large Domestic App',
       u'Bric-a-brac', u'Gas bottles', u'Bicycles', u'Soil',
       u'Plasterboard', u'Furniture', u'Other Scrap metal', u'Wood',
       u'Video tapes, DVDs and CDs', u'Paint', u'Car tyres'], dtype=object)

In [14]:
hwrcs_reu_ton.head()

,Authority,Period,QuestionNumber,QuText,RowText,ColText,Data,MaterialGroup
52,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,Books,Tonnage collected for reuse,7.86,Paper & Card
54,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,Textiles & footwear,Tonnage collected for reuse,36.624,Textiles
56,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,Rubble,Tonnage collected for reuse,1305.51,Rubble
157,Flintshire County Council,Oct 14 - Dec 14,Q016,Tonnes of material collected for recycling/reu...,Books,Tonnage collected for reuse,5.491,Paper & Card
159,Flintshire County Council,Oct 14 - Dec 14,Q016,Tonnes of material collected for recycling/reu...,Textiles & footwear,Tonnage collected for reuse,15.25,Textiles


In [20]:
hwrcs_reu_ton_la = hwrcs_reu_ton.groupby('Authority')['Data'].sum().to_frame().reset_index().rename(columns={'Data':'sum_dry_rec'})
hwrcs_reu_ton_la

,Authority,sum_dry_rec
0,Blaenau Gwent CBC,6.136
1,Cardiff County Council,320.310
2,Carmarthenshire County Council,19.020
3,Ceredigion County Council,1602.210
4,City and County of Swansea,271.600
5,Conwy CBC,18.620
6,Denbighshire County Council,213.945
7,Flintshire County Council,7708.283
8,Gwynedd Council,1175.870
9,Isle of Anglesey CC,9.920


In [15]:
hwrcs_rec_ton = question_16[(question_16.ColText == 'Tonnage collected for recycling') & (question_16.Data > 0)]
hwrcs_rec_ton.shape

(1899, 8)

In [17]:
hwrcs_rec_ton = question_16[(question_16.ColText == 'Tonnage collected for recycling')]
hwrcs_rec_ton.shape

(1910, 8)

In [20]:
hwrcs_reu_ton.head()

,Authority,Period,QuestionNumber,QuText,RowText,ColText,Data,MaterialGroup
52,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,Books,Tonnage collected for reuse,7.86,Paper & Card
54,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,Textiles & footwear,Tonnage collected for reuse,36.624,Textiles
56,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,Rubble,Tonnage collected for reuse,1305.51,Rubble
157,Flintshire County Council,Oct 14 - Dec 14,Q016,Tonnes of material collected for recycling/reu...,Books,Tonnage collected for reuse,5.491,Paper & Card
159,Flintshire County Council,Oct 14 - Dec 14,Q016,Tonnes of material collected for recycling/reu...,Textiles & footwear,Tonnage collected for reuse,15.25,Textiles


In [21]:
hwrcs_rec_ton.head()

,Authority,Period,QuestionNumber,QuText,RowText,ColText,Data,MaterialGroup
38,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,Green garden waste only,Tonnage collected for recycling,328.54,Organic
39,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,WEEE - Large Domestic App,Tonnage collected for recycling,241.51,WEEE
40,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,WEEE - Small Domestic App,Tonnage collected for recycling,117.48,WEEE
41,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,WEEE - Cathode Ray Tubes,Tonnage collected for recycling,44.59,WEEE
42,Flintshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,WEEE - Flourescent tubes and other light bulbs,Tonnage collected for recycling,1,WEEE


#Rejected send to residual

In [13]:
question_16[(question_16.ColText == 'Tonnage collected for recycling but actually rejected / disposed') & (question_16.Data > 0)]

,Authority,Period,QuestionNumber,QuText,RowText,ColText,Data,MaterialGroup
4514,Carmarthenshire County Council,Jan 15 - Mar 15,Q016,Tonnes of material collected for recycling/reu...,Paint,Tonnage collected for recycling but actually r...,32.2,Paint
4720,Carmarthenshire County Council,Oct 14 - Dec 14,Q016,Tonnes of material collected for recycling/reu...,Paint,Tonnage collected for recycling but actually r...,33.84,Paint
4940,Carmarthenshire County Council,Jul 14 - Sep 14,Q016,Tonnes of material collected for recycling/reu...,Paint,Tonnage collected for recycling but actually r...,57.98,Paint
5165,Carmarthenshire County Council,Apr 14 - Jun 14,Q016,Tonnes of material collected for recycling/reu...,Paint,Tonnage collected for recycling but actually r...,61.24,Paint
9669,Cardiff County Council,Apr 14 - Jun 14,Q016,Tonnes of material collected for recycling/reu...,Rubble,Tonnage collected for recycling but actually r...,18.32,Rubble


In [18]:
#This is making sure that the materials for reuse in hhkerb can be summed for sum_dry-recycling (not garden waste, etc.)
df = massflow_baseline.get_data()
hhkerb_rec = df[df.QuestionNumber == 'Q010']
hhkerb_reu_ton = hhkerb_rec[(hhkerb_rec.ColText == 'Tonnage Collected for Reuse') & (hhkerb_rec.Data > 0)]
#hhkerb_reu_ton = hhkerb_rec.groupby('ColText').get_group('Tonnage Collected for Reuse')
hhkerb_reu_ton.RowText.unique()
#hhkerb_reu_ton_qtr = hhkerb_reu_ton.pivot_table(values='Data', index=['Authority','Period'],columns='RowText', aggfunc = lambda x: x)

array([u'WEEE - Large Domestic App', u'Bric-a-brac',
       u'Textiles & footwear', u'Furniture', u'Wood',
       u'WEEE - Small Domestic App', u'Video tapes, DVDs and CDs'], dtype=object)

In [17]:
df = massflow_baseline.get_data()
hwrcs_rec = df[df.QuestionNumber == 'Q016']
hwrcs_reu_ton = hwrcs_rec.groupby('ColText').get_group('Tonnage collected for reuse')
hwrcs_reu_ton.RowText.unique()

array([u'Books', u'Textiles & footwear', u'Rubble',
       u'WEEE - Fridges & Freezers', u'WEEE - Large Domestic App',
       u'Bric-a-brac', u'Gas bottles', u'Bicycles', u'Soil',
       u'Plasterboard', u'Mixed Plastic Bottles',
       u'Green garden waste only', u'WEEE - Small Domestic App',
       u'WEEE - Flourescent tubes and other light bulbs',
       u'Composite food and beverage cartons', u'Mattresses',
       u'Mixed glass', u'Paper', u'Card', u'Mixed cans', u'Plastics',
       u'Co mingled materials', u'Wood', u'Other Scrap metal',
       u'Automotive batteries', u'Vegetable Oil', u'Mineral Oil', u'Paint',
       u'Furniture', u'WEEE - Cathode Ray Tubes', u'Carpets',
       u'Clear glass', u'Post consumer, non automotive batteries',
       u'Other materials', u'Mixed tyres', u'Video tapes, DVDs and CDs',
       u'Car tyres'], dtype=object)

In [19]:
#This is making sure that the materials for reuse in hwrcs can be summed for sum_dry-recycling (not garden waste, etc.)
hwrcs_reu_ton = question_16[(question_16.ColText == 'Tonnage collected for reuse') & (question_16.Data > 0)]
hwrcs_reu_ton.RowText.unique()

array([u'Books', u'Textiles & footwear', u'Rubble',
       u'WEEE - Fridges & Freezers', u'WEEE - Large Domestic App',
       u'Bric-a-brac', u'Gas bottles', u'Bicycles', u'Soil',
       u'Plasterboard', u'Furniture', u'Other Scrap metal', u'Wood',
       u'Video tapes, DVDs and CDs', u'Paint', u'Car tyres'], dtype=object)

In [21]:
df = massflow_baseline.get_hhkerb_rec_ton_qtr()
hhkerb_rec_ton_all = df.groupby('Authority').agg(np.sum)
hhkerb_rec_ton_all = hhkerb_rec_ton_all.reset_index()
hhkerb_rec_ton_all = hhkerb_rec_ton_all.drop(['Green garden waste only','Mixed garden and food waste',
                                              'Waste food only'],axis=1)
#Generate the sum of recycling materials (for most LAs with co-mingled recycling, it is just from 'Co mingled materials')
hhkerb_rec_ton_all['sum_dry_rec'] = hhkerb_rec_ton_all.sum(axis=1)

In [22]:
hhkerb_rec_ton_all.head()

RowText,Authority,Absorbent Hygiene Products (AHP),Aluminium cans,Car tyres,Card,Co mingled materials,Composite food and beverage cartons,Gas bottles,HDPE [2],Large vehicle tyres,Mixed Plastic Bottles,Mixed cans,Mixed glass,Mixed paper & card,Other Scrap metal,Other compostable waste,Other materials,PS [6],Paper,Plastics,"Post consumer, non automotive batteries",Steel cans,Textiles & footwear,Textiles only,"Video tapes, DVDs and CDs",WEEE - Fridges & Freezers,WEEE - Large Domestic App,WEEE - Small Domestic App,Wood,sum_dry_rec
0,Blaenau Gwent CBC,NaN,NaN,NaN,NaN,7111.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7111.50
1,Bridgend CBC,NaN,172.44,NaN,3194.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3206.88,NaN,NaN,NaN,NaN,NaN,2164.36,1498.68,1.5,513.14,11.98,NaN,NaN,NaN,NaN,NaN,NaN,10763.07
2,Caerphilly CBC,NaN,NaN,NaN,NaN,17828.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17828.17
3,Cardiff County Council,230.44,NaN,3.92,NaN,32416.59,NaN,8.94,8.98,0.63,NaN,NaN,NaN,NaN,NaN,4.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.56,11.73,NaN,NaN,32774.61
4,Carmarthenshire County Council,NaN,NaN,NaN,NaN,13012.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13012.69
